In [1]:


class PolynomialTerm:
    def __hash__(self):
        return hash(str(self))
    
    def __str__(self):
        return str(self.value)
    
    def is_number(self):
        return False
        
    def is_symbol(self):
        return False
    
    def could_extract_minus_sign(self):
        return False
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash(str(self))  
    
    def __eq__(self, other):
        if not isinstance(other, PolynomialTerm):
            return False
        return str(self) == str(other)  

class Variable(PolynomialTerm):
    def __init__(self, value):
        self.value = value
        
    def is_symbol(self):
        return True
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Power(self, exponent)
        elif isinstance(exponent, (int, float)):
            return Power(self, Constant(exponent))
        
    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        if not isinstance(other, Variable):
            return False
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __le__(self, other):
        return self.value <= other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __ge__(self, other):
        return self.value >= other.value
       


class Constant(PolynomialTerm):
    def __init__(self, value):
        self.value = float(value)
        
    def is_number(self):
        return True
        
    @property
    def is_integer(self):
        return self.value == int(self.value)
        
    def __gt__(self, second):
        if isinstance(second, Constant):
            return self.value > second.value
        return self.value > second
        
    def __eq__(self, second):
        if isinstance(second, Constant):
            return self.value == second.value
        return self.value == second
    
    def __hash__(self):
        return hash(self.value)
    
    def __mod__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value % other.value)
        return Constant(self.value % other)

    def __rmod__(self, other):
        return Constant(other % self.value)
    
    def __sub__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value - other.value)
        return Constant(self.value - other)

    def __rsub__(self, other):
        return Constant(other - self.value)
    
    def __pow__(self, exponent):
        if isinstance(exponent, Constant):
            return Constant(self.value ** exponent.value)
        return Constant(self.value ** exponent)
    
    def __floordiv__(self, other):
        if isinstance(other, Constant):
            return Constant(self.value // other.value)
        return Constant(self.value // other)

    def __rfloordiv__(self, other):
        return Constant(other // self.value)


class BinaryOperation(PolynomialTerm):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    @property
    def args(self):
        return [self.left, self.right]


class Addition(BinaryOperation):
    def __str__(self):
        return f"{self.left} + {self.right}"
    

class Subtraction(BinaryOperation):
    def __str__(self):
        return f"{self.left} - {self.right}"
    

class Multiplication(BinaryOperation):
    def __str__(self):
        # Adds parentheses around additions and subtractions
        left_str = str(self.left)
        right_str = str(self.right)

        if isinstance(self.left, (Addition, Subtraction)):
            left_str = f"({left_str})"
            
        if isinstance(self.right, (Addition, Subtraction)):
            right_str = f"({right_str})"
            
        return f"{left_str} * {right_str}"
    

class Power(BinaryOperation):
    def __str__(self):
        # Adds parentheses around the base for clarity
        left_str = str(self.left)
        if isinstance(self.left, (Addition, Subtraction, Multiplication)):
            left_str = f"({left_str})"
            
        return f"{left_str}^{self.right}"
    

class UnaryMinus(PolynomialTerm):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        term_str = str(self.value)
        if isinstance(self.value, (Addition, Subtraction, Multiplication)):
            term_str = f"({term_str})"
        return f"-{term_str}"
    
    def is_number(self):
        return self.value.is_number() if hasattr(self.value, 'is_number') else False
    
    def could_extract_minus_sign(self):
        return True
    
    @property
    def args(self):
        return [self.value]


class PolynomialParser:
    def __init__(self, expr_string):
        self.expr_tree = None
        self.parse(expr_string)

    
    def parse(self, expr):
        self.expr = expr.replace(' ', '') 
        self.pos = 0
        self.expr_tree = self.parse_expression()
        return self
    
    def current_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos]
    
    def next_char(self):
        if self.pos >= len(self.expr):
            return None
        return self.expr[self.pos + 1] if self.pos + 1 < len(self.expr) else None

    
    def parse_expression(self):
        if self.current_char() == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            # left = Multiplication(Constant(-1.0), self.parse_term())

            left = UnaryMinus(self.parse_term())
            
        else:
            left = self.parse_term()
        
        while self.current_char() in ('+', '-'):
            op = self.current_char()
            self.pos += 1
            right = self.parse_term()
            
            if op == '+':
                left = Addition(left, right)
            else:
                left = Subtraction(left, right)
        
        return left
    
    def parse_term(self):
        left = self.parse_factor()
        
        while self.current_char() == '*':
            self.pos += 1
            right = self.parse_factor()
            left = Multiplication(left, right)
                
        return left
    
    def parse_factor(self):
        char = self.current_char()
        
        # End of expression check
        if char is None:
            return None
        
        #  unary minus
        if char == '-' and self.next_char() and not self.next_char().isdigit():
            self.pos += 1
            return UnaryMinus(self.parse_factor())
        
        # Brackets
        if char == '(':
            self.pos += 1
            expr = self.parse_expression()
            self.pos += 1  # Skip closing bracket
            
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(expr, exponent)
                
            return expr
        
        #  numbers
        if char.isdigit() or (char == '-' and self.next_char() and self.next_char().isdigit()):
            return self.parse_number()
        
        #  variables
        if char.isalpha():
            var = self.parse_variable()
            
            # Check for power after variable
            if self.current_char() == '^':
                self.pos += 1
                exponent = self.parse_factor()
                return Power(var, exponent)

            return var
            
    
    def parse_number(self):
        start_pos = self.pos
        char = self.current_char()
        
        # Handle negative sign only at the beginning
        if char == '-':
            self.pos += 1
            char = self.current_char()
        
        # Parse digits and decimal point
        while char is not None and (char.isdigit() or char == '.'):
            self.pos += 1
            char = self.current_char()
        
        # Create constant
        value = float(self.expr[start_pos:self.pos])
        return Constant(value)
    
    def parse_variable(self):
        start_pos = self.pos
        char = self.current_char()
        
        while char is not None and char.isalnum():
            self.pos += 1
            char = self.current_char()
            
        variable = self.expr[start_pos:self.pos]
        return Variable(variable)
    

    def count(self, node=None):

        if node is None:
            node = self.expr_tree
        
        return self.count_terms(node)
    
    def count_terms(self, node):

        if type(node) in (Addition, Subtraction):
            left_count = self.count_terms(node.left)
            right_count = self.count_terms(node.right)
            return left_count + right_count
        else:
            return 1
        
    def find_variables(self, node = None, variables = None):

        if variables is None:
            variables = set()

        if node is None:
            node = self.expr_tree

        if isinstance(node, Variable):
            variables.add(node)

        elif isinstance(node, (Addition, Subtraction, Multiplication, Power)):
            self.find_variables(node.left, variables)
            self.find_variables(node.right, variables)
            
        elif isinstance(node, UnaryMinus):
            self.find_variables(node.value, variables)
    
        return variables
    

        
       


In [2]:
import pandas as pd
from IPython.display import display
import numpy as np
from itertools import combinations

def try_factorising(term, co_kernel):
    print("Term: ", term)
    print("Cokernel: ", co_kernel)

    if equal_terms(term, co_kernel):
        return Constant(1.0)
    
    if co_kernel == Constant(1):
        return term

    
    if type(term) == UnaryMinus and type(co_kernel) != UnaryMinus:
        inner_result = try_factorising(term.value, co_kernel)
        # if inner_result is not None:
        return UnaryMinus(inner_result)
        # else:
        #     return None
        
    if type(co_kernel) == UnaryMinus and type(term) != UnaryMinus:
        inner_result = try_factorising(term, co_kernel.value)
        # if inner_result is not None:
        return UnaryMinus(inner_result)
        # else:
        #     return None
        
    if type(term) == UnaryMinus and type(co_kernel) == UnaryMinus:
        return try_factorising(term.value, co_kernel.value)


    # Power factorisation
    if type(term) == Power and type(co_kernel) == Power:
        if equal_terms(term.left, co_kernel.left) and term.right.value > co_kernel.right.value:
            new_power = term.right.value - co_kernel.right.value
            if new_power == 1:
                return term.left 
            else:
                Power(term.left, Constant(new_power))
    

    if type(term) == Power and type(co_kernel) == Variable:
        if equal_terms(term.left, co_kernel) and term.right.value > 1:
            new_power = term.right.value - 1
            if new_power == 1:
                return term.left 
            else:
                Power(term.left, Constant(new_power))


    
    if type(term) == Multiplication:
        return factorise_multiplication(term, co_kernel)
    
    return None


def factorise_multiplication(term, co_kernel):
    term_factors = flatten_multiplication_factors(term)
    
    if type(co_kernel) == Multiplication:
        return factorise_with_multiple_factors(term_factors, flatten_multiplication_factors(co_kernel))
    else:
        return factorise_with_single_factor(term_factors, co_kernel)


def factorise_with_multiple_factors(term_factors, cokernel_factors):
    remaining_factors = term_factors.copy()  
    
    for factor in cokernel_factors:
        remaining_factors, in_list = remove_factor_from_list(remaining_factors, factor)

        if not in_list:
            return None

    return build_result_from_factors(remaining_factors)


def factorise_with_single_factor(term_factors, co_kernel):
    remaining_factors = term_factors.copy() 
    
    remaining_factors, in_list = remove_factor_from_list(remaining_factors, co_kernel)

    if not in_list:
        return None
    
    return build_result_from_factors(remaining_factors)
    
    # return None


def remove_factor_from_list(factors, target):

    for i, factor in enumerate(factors):
        # Exact match
        if equal_terms(factor, target):
            factors.pop(i)
            return factors, True
        
        # Powers need to have the same base
        if (type(factor) == Power and type(target) == Power and
            equal_terms(factor.left, target.left) and
            type(factor.right) == Constant and type(target.right) == Constant and factor.right.value > target.right.value):
            
            new_power = factor.right.value - target.right.value
            if new_power == 1:
                new_factor = factor.left
            else:
                new_factor = Power(factor.left, Constant(new_power))
            
            factors.pop(i)
            factors.insert(i, new_factor)
            return factors, True
        
        # Power and variable need to have the same base
        if (type(factor) == Power and type(target) == Variable and equal_terms(factor.left, target) and
            type(factor.right) == Constant):
            
            new_power = factor.right.value - 1
            if new_power == 1:
                new_factor = factor.left
            else:
                new_factor = Power(factor.left, Constant(new_power))
            
            factors.pop(i)
            factors.insert(i, new_factor)
            return factors, True
    
    return factors, False


def build_result_from_factors(factors):

    if len(factors) == 0:
        return Constant(1.0)
    elif len(factors) == 1:
        return factors[0]
    else:
        result = factors[0]
        for factor in factors[1:]:
            result = Multiplication(result, factor)
        return result
    



def flatten_multiplication_factors(expr):
    if type(expr) != Multiplication:
        return [expr]
    
    factors = []
    factors.extend(flatten_multiplication_factors(expr.left))
    factors.extend(flatten_multiplication_factors(expr.right))
    return factors



def normalise_multiplication_factors(factors):

    def factor_sort(factor):

        if type(factor) == Constant:
            return (0, factor.value, 0)
        
        elif type(factor) == Variable:
            return (1, factor.value, 0)
        
        elif type(factor) == Power:
            if type(factor.left) == Variable and type(factor.right) == Constant:
                return (2, factor.left.value, factor.right.value)
            
            elif type(factor.left) == Variable:
                return (2, factor.left.value, float('inf'))
            
            else:
                return (3, str(factor), 0)
            
        else:
            return (4, str(factor), 0)
        
    
    return sorted(factors, key=factor_sort)



def equal_terms(term1, term2):
    if type(term1) != type(term2):
        return False
    
    if type(term1) == Variable:
        return term1.value == term2.value
    
    elif type(term1) == Constant:
        return (abs(term1.value - term2.value) < 1e-10)
    
    elif type(term1) == Multiplication:
        factors1 = flatten_multiplication_factors(term1)
        factors2 = flatten_multiplication_factors(term2)
        
        if len(factors1) != len(factors2):
            return False
        
        norm_factors1 = normalise_multiplication_factors(factors1)
        norm_factors2 = normalise_multiplication_factors(factors2)
        
        return all(equal_terms(f1, f2) for f1, f2 in zip(norm_factors1, norm_factors2))
    
    elif type(term1) in (Addition, Subtraction, Power):
        left_equals = equal_terms(term1.left, term2.left)
        right_equals = equal_terms(term1.right, term2.right)
        return (left_equals and right_equals)
    
    elif type(term1) == UnaryMinus:
        return equal_terms(term1.value, term2.value)


def extract_terms(node, terms=None):
    if terms is None:
        terms = []
    
    if type(node) == Addition:
        extract_terms(node.left, terms)
        extract_terms(node.right, terms)
        
    elif type(node) == Subtraction:
        extract_terms(node.left, terms)

        extract_terms(UnaryMinus(node.right), terms)

    elif type(node) == UnaryMinus:
        terms.append(node)

    else:
        terms.append(node)
    
    return terms


def create_polynomial_matrix(expression_tree):
    full_terms = extract_terms(expression_tree)

    # print("Full terms", full_terms)
    
    # Calls class to find all variables
    parser = PolynomialParser("")
    parser.expr_tree = expression_tree
    all_variables = sorted(list(parser.find_variables()))

    # print(type(parser.find_variables()[0]))
    
    matrix_rows = []
    for i, original_term in enumerate(full_terms):
        variables = {}
        term_sign = 1
        coefficient = 1

        # print(original_term)
        
        # Check if term is negative
        if type(original_term) == UnaryMinus:
            term_sign = -1
            inner_term = original_term.value

        else:
            inner_term = original_term

        # Term is a constant
        if type(inner_term) == Constant:
            coefficient = inner_term.value

        else:

            if type(inner_term) == Multiplication:
                factors = flatten_multiplication_factors(inner_term)

                # Coefficient of multiplication term
                coefficient = [f for f in factors if type(f) == Constant]
                coefficient = coefficient[0].value

 
            # Tries to factor out the biggest combination of variables from the term
            for var in all_variables:
                # var_node = Variable(var)
                power = 0
                current_term = inner_term

                # print("before lopp: ", var, type(var))
                
                while True:
                    # print(current_term, var)
                    factored_result = try_factorising(current_term, var)

                    if factored_result is not None:
                        power += 1
                        current_term = factored_result
                    else:
                        break
                
                if power > 0:
                    variables[var] = power
        
        # print(inner_term, variables)
        
        # Keeps track position of the original term in the original expression. Important for later on
        row = {'term_id': i}

        for var in all_variables:
            # print(var)
            if var in variables:
                row[var] = variables[var]
            else:
                row[var] = 0

        row['coefficient'] = abs(coefficient)

        if coefficient >= 0:
            row['sign'] = term_sign
        else:
            row['sign'] = 0

        row['original_term'] = original_term

        matrix_rows.append(row)

    matrix_df = pd.DataFrame(matrix_rows)

    # display(matrix_df)

    return matrix_df, all_variables



def largest_common_cube(matrix_rows, variables):

    # In a specific row finds the cube that is common in the whole term (a row represents a single term)
    common_cube = {}

    for var in variables:

        var_numbers = [row[var] for row in matrix_rows]
        # print(var_numbers)
        min_power = min(var_numbers)
        # print(min_power)

        if min_power > 0:
            common_cube[var] = min_power
    
    print(common_cube)
    return common_cube



def divide_rows_by_cube(matrix_rows, cube_divisor, variables):

    row_res = []

    for row in matrix_rows:

        divided_row = row.copy()

        # Cube divisor contains all variables as keys and their powers
        for var in list(cube_divisor.keys()):

            # print(divided_row, type(var), type(list(cube_divisor.keys())[0]))

            # Divides by subtracting powers
            if var in cube_divisor:
                divided_row[var] = row[var] - cube_divisor[var]
            else:
                divided_row[var] = row[var] - 0
            
            # print(divided_row)

        row_res.append(divided_row)


    return row_res



def cube_free(matrix_rows, variables):

    print(len(matrix_rows), variables)

    # Kernel should have more than one term
    if len(matrix_rows) <= 1:
        return True
    
    common_cube = largest_common_cube(matrix_rows, variables)

    if len(common_cube) == 0:
        return True
    else:
        return False


# The cube is treated as a dictionary and it needs to be converted to a term
def cube_dict_to_term(cube_dict, variables):

    # print("CUBE DICT:", cube_dict)
    if not cube_dict:
        return Constant(1)
    
    factors = []
    sorted_vars = sorted(variables)

    for var in sorted_vars:

        if var in cube_dict and cube_dict[var] > 0:
            if cube_dict[var] == 1:
                factors.append(Variable(var))

            else:
                factors.append(Power(Variable(var), Constant(cube_dict[var])))
    
    if len(factors) == 0:
        return Constant(1)
    
    elif len(factors) == 1:
        return factors[0]
    
    else:
        result = factors[0]

        for factor in factors[1:]:
            result = Multiplication(result, factor)

        return result



def rows_to_expression(matrix_rows, variables):

    if len(matrix_rows) == 0:
        return Constant(0)
    
    terms = []
    for row in matrix_rows:
        term_factors = []
        
        if row['coefficient'] != 1:
            term_factors.append(Constant(row['coefficient']))
        
        for var in variables:
            if row[var] > 0:
                if row[var] == 1:
                    term_factors.append(Variable(var))

                else:
                    term_factors.append(Power(Variable(var), Constant(row[var])))

        
        if len(term_factors) == 0:
            term = Constant(row['coefficient'])

        elif len(term_factors) == 1:
            term = term_factors[0]

        else:
            term = term_factors[0]
            for factor in term_factors[1:]:
                term = Multiplication(term, factor)
        
        if row['sign'] < 0:
            term = UnaryMinus(term)
        
        terms.append(term)
    
    if len(terms) == 1:
        return terms[0]
    
    else:
        result = terms[0]
        for term in terms[1:]:
            if type(term) == UnaryMinus:
                result = Subtraction(result, term.value)

            else:
                result = Addition(result, term)

        return result




def recursively_extract_kernels(matrix_rows, variables, var_index = 0, acc_cokernel = None):
    kernels_cokernels = []
    
    if acc_cokernel is None:
        acc_cokernel = {}
    
    # Converts pandas to dictionary of rows
    # matrix_rows = matrix_df.to_dict('records')
    
    # Stops if gone through all variables or there are less than two terms in a potential kernel
    if var_index >= len(variables) or len(matrix_rows) < 2:
        return kernels_cokernels
    
    sel_var = variables[var_index]
    
    
    rows_with_var = [row for row in matrix_rows if row[sel_var] > 0]
    
    # There need to be at leats two terms
    if len(rows_with_var) > 1:
        common_cube = largest_common_cube(rows_with_var, variables)
        # print("Commmon cube: ", common_cube, sel_var)

        # The cube should the have the current variables
        if sel_var in common_cube:

            # Remove cube from the rows. Cube is being factored out
            divided_rows = divide_rows_by_cube(rows_with_var, common_cube, variables)
            
            # Check if the current variable can no longer be removed from the terms
            # Check that there are at least 2 terms
            if cube_free(divided_rows, variables) and len(divided_rows) > 1:

                new_cokernel = acc_cokernel.copy()

                for var, power in common_cube.items():

                    # Add the powers of the variables to the dictionary
                    if var in new_cokernel:
                        new_cokernel[var] = new_cokernel[var] + power
                    else:
                        new_cokernel[var] = power

                
                cokernel = cube_dict_to_term(new_cokernel, variables)
                kernel = rows_to_expression(divided_rows, variables)
                
                kernel_cokernel_pair = {
                    'cokernel_dict': new_cokernel,
                    'kernel_rows': divided_rows,
                    'cokernel': cokernel,
                    'kernel': kernel}
                
                kernels_cokernels.append(kernel_cokernel_pair)
                

                # kernel_df = pd.DataFrame(divided_rows)

                # Goes through recursion to find kernels of selected kernel
                sub_kernels = recursively_extract_kernels(divided_rows, variables, 0, new_cokernel)
                kernels_cokernels.extend(sub_kernels)
    
    next_kernels = recursively_extract_kernels(matrix_rows, variables, var_index + 1, acc_cokernel)

    kernels_cokernels.extend(next_kernels)
    
    return kernels_cokernels



def find_all_kernels_cokernels(expression_str):
    print(f"Original expression: {expression_str}")
    print("=" * 80)
    
    parser = PolynomialParser(expression_str)
    expression_tree = parser.expr_tree
    
    matrix_df, variables = create_polynomial_matrix(expression_tree)
    
    
    display(matrix_df)
    matrix_rows = matrix_df.to_dict('records')
    
    kernels_cokernels = [{
        'cokernel': {},
        'kernel_rows': matrix_rows,
        'cokernel': Constant(1),
        'kernel': expression_tree
    }]
    
    
    extracted_kernels = recursively_extract_kernels(matrix_rows, variables)
    kernels_cokernels.extend(extracted_kernels)
    
    unique_kernels = []
    seen_pairs = set()
    
    # Last check to avoid duplicates
    for item in kernels_cokernels:
        kernel_co_pair = (str(item['cokernel']), str(item['kernel']))

        if kernel_co_pair not in seen_pairs:
            seen_pairs.add(kernel_co_pair)
            unique_kernels.append(item)
    
    return unique_kernels, variables




def display_results(kernels_cokernels):
    print(f"Found {len(kernels_cokernels)} kernel-cokernel pairs:")
    print("=" * 80)
    
    for i, item in enumerate(kernels_cokernels):
        print(f"Pair {i+1}:")
        print(f"  Cokernel: {str(item['cokernel'])}")
        print(f"  Kernel:   {str(item['kernel'])}")
        



def convert_row_to_term(row, variables):

    coeff = row['coefficient']
    sign = row['sign']
    coefficient = coeff * sign
    
    # Construct the different factors
    var_factors = []
    for var in sorted(variables):
        if row[var] > 0:
            if row[var] == 1:
                var_factors.append(Variable(var))
            else:
                var_factors.append(Power(Variable(var), Constant(row[var])))
    


    # Combine with coefficient
    if coefficient == 1 and var_factors:

        if len(var_factors) == 1:
            return var_factors[0]
        else:
            result = var_factors[0]

            for factor in var_factors[1:]:
                result = Multiplication(result, factor)
            return result
        
    elif coefficient == -1 and var_factors:
        if len(var_factors) == 1:
            return UnaryMinus(var_factors[0])
        
        else:
            result = var_factors[0]
            for factor in var_factors[1:]:
                result = Multiplication(result, factor)

            return UnaryMinus(result)
        
    elif coefficient != 1 and var_factors:
        coeff_factor = Constant(abs(coefficient))
        result = coeff_factor

        for factor in var_factors:
            result = Multiplication(result, factor)

        if coefficient < 0:
            return UnaryMinus(result)
        else:
            return result
        
    else:
        return Constant(coefficient)




def create_kernel_cube_matrix(kernels_cokernels, variables):
    
    if len(kernels_cokernels) == 0:
        print("No kernels found")
        return pd.DataFrame()
    

    cube_order = []
    seen_cubes = set()
    
    # Add terms of the kernels in they order that they appear

    for item in kernels_cokernels:
        for row in item['kernel_rows']:
            cube_term = convert_row_to_term(row, variables)
            # cube_str = str(cube_term)
            
            if str(cube_term) not in seen_cubes:
                cube_order.append(cube_term)
                seen_cubes.add(str(cube_term))
            

    # print(f"Found {len(cube_order)} unique cubes in order: {[str(c) for c in cube_order]}")

    # # Sort kernels by cokernel complexity
    # def cokernel_sort_key(cokernel):
    #     cokernel_str = str(cokernel['cokernel'])
    #     if cokernel_str == "1":
    #         return (0, "")
    #     else:
    #         return (1, len(cokernel_str), cokernel_str)

    # sorted_kernels = sorted(kernels_cokernels, key=cokernel_sort_key)

    # sorted_kernels = kernels_cokernels

    kcm_matrix = []
    
    for item in kernels_cokernels:
        row_data = {'cokernel': item['cokernel']}  

        # Track the original term ids of the kernel
        kernel_cubes = {} 
        
        for row in item['kernel_rows']:
            cube_term = convert_row_to_term(row, variables)
            # print(cube_term)
            
            # Track the term_id for this cube
            if str(cube_term) not in kernel_cubes:
                kernel_cubes[str(cube_term)] = set()
            
            kernel_cubes[str(cube_term)].add(row['term_id'])
            # print(row['term_id'])
        
        for cube_term in cube_order:
            cube_str = str(cube_term)
            if cube_str in kernel_cubes:
                term_ids = sorted(list(kernel_cubes[cube_str]))
                if term_ids:
                    row_data[cube_term] = f"1({','.join(map(str, term_ids))})"
                else:
                    row_data[cube_term] = "1"
            else:
                row_data[cube_term] = "0"
        
        kcm_matrix.append(row_data)
        
        # print(f"Kernel {str(item['cokernel'])}: cubes = {list(kernel_cubes.keys())}")
    kcm_matrix = pd.DataFrame(kcm_matrix)
    return kcm_matrix




def analyse_polynomial(expression_str):
    kernels_cokernels, variables = find_all_kernels_cokernels(expression_str)
    
    display_results(kernels_cokernels)
    
    print("\n" + "="*80)
    print("KERNEL-CUBE MATRIX (KCM)")
    print("="*80)


    kcm_matrix = create_kernel_cube_matrix(kernels_cokernels, variables)
    
    display(kcm_matrix)
    
    return kcm_matrix, kernels_cokernels, variables



test_expr = "-(4*x^6*y*z^2) - (2*y*x^4*z^3) - (3*z^4*y) + (5*x^2*y^2) + (11*x^2)-(14*x^3*y) - (29*y*x^2*z^7) - (34*z*y) + (50*x*y^2) + (90*y*x*z^2)"

print("POLYNOMIAL:")
print("=" * 80)
kcm_matrix_poly, kernels_cokernels, variables = analyse_polynomial(test_expr)

print("\n" + "=" * 80)
print("SIN(X) TAYLOR SERIES:")
print("=" * 80)

sin_expr = "x - 3*x^3 + 5*x^5 - 7*x^7"
kcm_matrix_sin, kernels_cokernels, variables = analyse_polynomial(sin_expr)



POLYNOMIAL:
Original expression: -(4*x^6*y*z^2) - (2*y*x^4*z^3) - (3*z^4*y) + (5*x^2*y^2) + (11*x^2)-(14*x^3*y) - (29*y*x^2*z^7) - (34*z*y) + (50*x*y^2) + (90*y*x*z^2)
Term:  4.0 * x^6.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x^5.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x^4.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x^3.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x^2.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x * y * z^2.0
Cokernel:  x
Term:  4.0 * y * z^2.0
Cokernel:  x
Term:  4.0 * x^6.0 * y * z^2.0
Cokernel:  y
Term:  4.0 * x^6.0 * z^2.0
Cokernel:  y
Term:  4.0 * x^6.0 * y * z^2.0
Cokernel:  z
Term:  4.0 * x^6.0 * y * z
Cokernel:  z
Term:  4.0 * x^6.0 * y
Cokernel:  z
Term:  2.0 * y * x^4.0 * z^3.0
Cokernel:  x
Term:  2.0 * y * x^3.0 * z^3.0
Cokernel:  x
Term:  2.0 * y * x^2.0 * z^3.0
Cokernel:  x
Term:  2.0 * y * x * z^3.0
Cokernel:  x
Term:  2.0 * y * z^3.0
Cokernel:  x
Term:  2.0 * y * x^4.0 * z^3.0
Cokernel:  y
Term:  2.0 * x^4.0 * z^3.0
Cokernel:  y
Term:  2.0 * y * x^4.0 * z^3.0
Cokernel:

,term_id,x,y,z,coefficient,sign,original_term
0,0,6,1,2,4.0,-1,-(4.0 * x^6.0 * y * z^2.0)
1,1,4,1,3,2.0,-1,-(2.0 * y * x^4.0 * z^3.0)
2,2,0,1,4,3.0,-1,-(3.0 * z^4.0 * y)
3,3,2,2,0,5.0,1,5.0 * x^2.0 * y^2.0
4,4,2,0,0,11.0,1,11.0 * x^2.0
5,5,3,1,0,14.0,-1,-(14.0 * x^3.0 * y)
6,6,2,1,7,29.0,-1,-(29.0 * y * x^2.0 * z^7.0)
7,7,0,1,1,34.0,-1,-(34.0 * z * y)
8,8,1,2,0,50.0,1,50.0 * x * y^2.0
9,9,1,1,2,90.0,1,90.0 * y * x * z^2.0


{x: 1}
8 [x, y, z]
{}
{x: 1}
6 [x, y, z]
{}
{x: 1, y: 1}
3 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{y: 1}
5 [x, y, z]
{}
{x: 1}
3 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{z: 2}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{}
{z: 1}
2 [x, y, z]
{}
{y: 1, z: 2}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{}
{z: 1}
2 [x, y, z]
{}
{y: 1}
7 [x, y, z]
{}
{x: 1}
5 [x, y, z]
{}
{x: 1}
3 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{z: 2}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{}
{z: 1}
2 [x, y, z]
{}
{y: 1}
2 [x, y, z]
{}
{z: 2}
4 [x, y, z]
{}
{x: 1}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{}
{z: 1}
2 [x, y, z]
{}
{x: 1, z: 1}
2 [x, y, z]
{}
{y: 1, z: 2}
4 [x, y, z]
{}
{x: 1}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{}
{z: 1}
2 [x, y, z]
{}
{x: 1, z: 1}
2 [x, y, z]
{}
{y: 1}
9 [x, y, z]
{}
{x: 1}
7 [x, y, z]
{}
{x: 1}
5 [x, y, z]
{}
{x: 1}
3 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{x: 1, z: 2}
2 [x, y, z]
{}
{z: 2}
3 [x, y, z]
{}
{x: 2}
2 [x, y, z]
{

,cokernel,-(4.0 * x^6.0 * y * z^2.0),-(2.0 * x^4.0 * y * z^3.0),-(3.0 * y * z^4.0),5.0 * x^2.0 * y^2.0,11.0 * x^2.0,-(14.0 * x^3.0 * y),-(29.0 * x^2.0 * y * z^7.0),-(34.0 * y * z),50.0 * x * y^2.0,...,-(4.0 * x^6.0),-(2.0 * x^4.0 * z),-(3.0 * z^2.0),-(29.0 * x^2.0 * z^5.0),90.0 * x,-(2.0 * x^4.0),-(3.0 * z),-(29.0 * x^2.0 * z^4.0),-3.0,-(29.0 * x^2.0 * z^3.0)
0,1.0,1(0),1(1),1(2),1(3),1(4),1(5),1(6),1(7),1(8),...,0,0,0,0,0,0,0,0,0,0
1,x,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,x^2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,x^3.0 * y,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,x^4.0 * y * z^2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,x^2.0 * y,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,x^2.0 * y * z^2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,x^2.0 * y * z^3.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,x * y,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,x * y^2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



SIN(X) TAYLOR SERIES:
Original expression: x - 3*x^3 + 5*x^5 - 7*x^7
Term:  x
Cokernel:  x
Term:  1.0
Cokernel:  x
Term:  3.0 * x^3.0
Cokernel:  x
Term:  3.0 * x^2.0
Cokernel:  x
Term:  3.0 * x
Cokernel:  x
Term:  3.0
Cokernel:  x
Term:  5.0 * x^5.0
Cokernel:  x
Term:  5.0 * x^4.0
Cokernel:  x
Term:  5.0 * x^3.0
Cokernel:  x
Term:  5.0 * x^2.0
Cokernel:  x
Term:  5.0 * x
Cokernel:  x
Term:  5.0
Cokernel:  x
Term:  7.0 * x^7.0
Cokernel:  x
Term:  7.0 * x^6.0
Cokernel:  x
Term:  7.0 * x^5.0
Cokernel:  x
Term:  7.0 * x^4.0
Cokernel:  x
Term:  7.0 * x^3.0
Cokernel:  x
Term:  7.0 * x^2.0
Cokernel:  x
Term:  7.0 * x
Cokernel:  x
Term:  7.0
Cokernel:  x


,term_id,x,coefficient,sign,original_term
0,0,1,1.0,1,x
1,1,3,3.0,-1,-(3.0 * x^3.0)
2,2,5,5.0,1,5.0 * x^5.0
3,3,7,7.0,-1,-(7.0 * x^7.0)


{x: 1}
4 [x]
{}
{x: 2}
3 [x]
{}
{x: 2}
2 [x]
{}
Found 4 kernel-cokernel pairs:
Pair 1:
  Cokernel: 1.0
  Kernel:   x - 3.0 * x^3.0 + 5.0 * x^5.0 - 7.0 * x^7.0
Pair 2:
  Cokernel: x
  Kernel:   1.0 - 3.0 * x^2.0 + 5.0 * x^4.0 - 7.0 * x^6.0
Pair 3:
  Cokernel: x^3.0
  Kernel:   -3.0 + 5.0 * x^2.0 - 7.0 * x^4.0
Pair 4:
  Cokernel: x^5.0
  Kernel:   5.0 - 7.0 * x^2.0

KERNEL-CUBE MATRIX (KCM)


,cokernel,x,-(3.0 * x^3.0),5.0 * x^5.0,-(7.0 * x^7.0),1.0,-(3.0 * x^2.0),5.0 * x^4.0,-(7.0 * x^6.0),-3.0,5.0 * x^2.0,-(7.0 * x^4.0),5.0,-(7.0 * x^2.0)
0,1.0,1(0),1(1),1(2),1(3),0,0,0,0,0,0,0,0,0
1,x,0,0,0,0,1(0),1(1),1(2),1(3),0,0,0,0,0
2,x^3.0,0,0,0,0,0,0,0,0,1(1),1(2),1(3),0,0
3,x^5.0,0,0,0,0,0,0,0,0,0,0,0,1(2),1(3)
